In [1]:
from langchain.llms import LlamaCpp
from langchain import PromptTemplate, LLMChain
import warnings
warnings.filterwarnings('ignore')

In [2]:
model_path = "/home/voris/edu/project-rag/models/llm/gemma-2-2b-it-GGUF/gemma-2-2b-it-Q4_K_M.gguf"

llm = LlamaCpp(
    model_path=model_path,
    temperature=0.7,
    top_k=40,
    top_p=0.9,
    max_tokens=4096,  
    n_ctx=4096,
    model_kwargs={
        'frequency_penalty': 0,
        'presence_penalty': 0
    },
    verbose=False
)

llama_new_context_with_model: n_batch is less than GGML_KQ_MASK_PAD - increasing to 32
llama_new_context_with_model: n_ctx_per_seq (4096) < n_ctx_train (8192) -- the full capacity of the model will not be utilized


In [4]:
prompt_template = """
Answer the following question based only on the provided context. 
If the information is not available in the context, reply with "I don't know."
After each answer, provide a brief explanation of how you arrived at that answer.

Context:
{context}

Question:
{question}

Answer:
"""

prompt = PromptTemplate(
    input_variables=["context", "question"],
    template=prompt_template,
)

In [5]:
llm_chain = LLMChain(llm=llm, prompt=prompt)

In [6]:
question = "what is territorial sea?"
context = "The territorial sea is the belt of coastal waters extending at most 12 nautical miles from the baseline of a coastal state."
# context = context['documents'][0]

In [7]:
response = llm_chain.run({"context": context, "question": question})

print(response)

The territorial sea is the belt of coastal waters extending at most 12 nautical miles from the baseline of a coastal state.

Explanation:
This answer directly answers the question based on the provided definition. The context states that the territorial sea is defined as a "belt of coastal waters extending at most 12 nautical miles from the baseline".


Let me know if you'd like to continue with other questions! 😊 



In [7]:

from langchain.text_splitter import RecursiveCharacterTextSplitter


splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)


In [8]:
from langchain_community.document_loaders import PDFPlumberLoader

loader = PDFPlumberLoader(file_path="../data/pdfs/unclos_e.pdf")
docs = loader.load()


In [9]:
pdf_splits = splitter.split_documents(docs)

In [ ]:
# Store documents in ChromaDB
for i, doc in enumerate(pdf_splits):
    vector = tfidf_matrix[i].toarray().flatten()
    collection.add(
        ids=[str(i)],
        documents=doc.page_content,
        metadatas=[doc.metadata],
        embeddings=[vector.tolist()]
    )

# Function to retrieve most relevant documents based on user query
def retrieve_relevant_docs(query, top_n=10):
    query_vector = vectorizer.transform([query]).toarray().flatten()
    results = collection.query(query_embeddings=[query_vector.tolist()], n_results=top_n)
    return results

In [3]:
from chromadb import PersistentClient, Settings
persist_directory = '../models/embeddings/chromadb'
anclient = PersistentClient(path=persist_directory, settings=Settings(allow_reset=True))

In [2]:
from embedding_model import MyEmbeddings

/home/voris/edu/project-rag/.venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [6]:
collection = anclient.get_or_create_collection('unclos_se')

In [11]:
for i, doc in enumerate(pdf_splits):
    collection.add(
        ids=[str(i)],
        documents=doc.page_content,
        metadatas=[doc.metadata],
        embeddings=MyEmbeddings(model_path="../models/embedding_model/all-mpnet-base-v2")
    )

/home/voris/edu/project-rag/.venv/lib/python3.12/site-packages/torch/cuda/__init__.py:128: UserWarning: CUDA initialization: CUDA unknown error - this may be due to an incorrectly set up environment, e.g. changing env variable CUDA_VISIBLE_DEVICES after program start. Setting the available devices to be zero. (Triggered internally at ../c10/cuda/CUDAFunctions.cpp:108.)
  return torch._C._cuda_getDeviceCount() > 0


ValueError: Expected embeddings to be a list of floats or ints, a list of lists, a numpy array, or a list of numpy arrays, got <embedding_model.MyEmbeddings object at 0x7b9fbee61e50> in add.